In [1]:
from langchain_community.document_loaders import PyPDFLoader,PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA



In [5]:
loader=PyPDFDirectoryLoader("./us_census")
docs=loader.load()

docs=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=150).split_documents(docs)
docs[0]


Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0})

In [4]:
embed=HuggingFaceBgeEmbeddings(
    model_name='BAAI/bge-small-en-v1.5', # sentence-transformers/
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)


c:\Users\kanis\LLM\Ollama\langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\kanis\LLM\Ollama\langchain\venv\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kanis\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to se

In [7]:
embed.embed_query(docs[0].page_content)

[-0.08465687930583954,
 -0.011909916996955872,
 -0.033789291977882385,
 0.02945590578019619,
 0.051915980875492096,
 0.057384002953767776,
 -0.04100174456834793,
 0.02742678113281727,
 -0.105128213763237,
 -0.01580561324954033,
 0.07948585599660873,
 0.0564318411052227,
 -0.013176549226045609,
 -0.034154411405324936,
 0.005816041491925716,
 0.047254811972379684,
 -0.013074689544737339,
 0.003129902994260192,
 -0.03442254662513733,
 0.03084062784910202,
 -0.04090859740972519,
 0.03527381643652916,
 -0.024376146495342255,
 -0.04358315467834473,
 0.024150321260094643,
 0.013198665343225002,
 -0.004844519775360823,
 0.019234763458371162,
 -0.0543912798166275,
 -0.14273501932621002,
 0.005155278369784355,
 0.02931155636906624,
 -0.05608109384775162,
 -0.008535378612577915,
 0.031414154917001724,
 0.02767363376915455,
 -0.02061881497502327,
 0.082423135638237,
 0.041542526334524155,
 0.05796552076935768,
 -0.03715868666768074,
 0.006261579692363739,
 -0.02413901500403881,
 -0.005617948714643

In [8]:
db=FAISS.from_documents(docs[:100],embed)

In [9]:
query="HOW POVERTY IS MEASURED?"
response=db.similarity_search(query)

for i in response:
    print(i.page_content)

a family’s or individual’s income 
is to their poverty threshold. It measures the depth of poverty for those with incomes below their poverty threshold.
In this brief, the income-to-poverty 
ratio is reported as both a per-
centage and ratio. For example, 
an income-to-poverty-ratio of 125 percent indicates families or individuals with income equal to 1.25 times their poverty threshold, while an income-to-poverty ratio of 50 percent identifies families or individuals with income equal to one-half of their poverty threshold. 
Families and individuals identified 
as in poverty and described in the preceding sections of this brief had an income-to-poverty ratio of less than 100 percent.
14 For more information, refer to  
<www.census.gov/geographies/reference-
maps/2010/geo/2010-census-regions-and-
divisions-of-the-united-states.html>.Table 1 for states and Table 2 for
Poverty in States and Metropolitan  
Areas: 2022
American Community Survey Briefs
December 2023ACSBR-016By Craig Benson
I

In [10]:
query="WHAT IS HEALTH INSURANCE COVERAGE?"
response=db.similarity_search(query)

for i in response:
    print(i.page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.
or a union, coverage purchased dire

In [11]:
retriever=db.as_retriever(search_type="similarity",search_kwargs={'k':3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FCDE6BE500> search_kwargs={'k': 3}


In [12]:
retriever1=db.as_retriever(search_type="similarity_score_threshold",search_kwargs={'k':4,'score_threshold':0.79})
print(retriever1)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FCDE6BE500> search_type='similarity_score_threshold' search_kwargs={'k': 4, 'score_threshold': 0.79}


In [36]:
import os
HUGGINGFACEHUB_API_TOKEN=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [46]:
from langchain_community.llms import HuggingFaceHub,HuggingFaceEndpoint
from langchain.chains import LLMChain
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, model_kwargs={'max_length':128, 'temperature':0.5, 'token':HUGGINGFACEHUB_API_TOKEN},
)
llm_chain = LLMChain(llm=llm)
print(llm_chain.run(query))

ValidationError: 1 validation error for HuggingFaceEndpoint
__root__
  Parameters {'temperature'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter. (type=value_error)

In [47]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

c:\Users\kanis\LLM\Ollama\langchain\venv\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kanis\.cache\huggingface\hub\models--mistralai--Mistral-7B-v0.1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


KeyboardInterrupt: 

In [48]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [49]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])


In [50]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)


query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [ ]:
result = retrievalQA.invoke({"query": query})
print(result['result'])